# Extracción de características

In [3]:
import numpy as np                # Numpy nunca puede faltar
import pandas as pd               # El siempre confiable Pandas
from os import listdir, path      # Para leer/escribir archivos del sistema

import librosa                    # Se usa para análisis de canciones pero hay una cosa que nos viene bien de aquí

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import statsmodels as sm
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa import stattools

import librosa
from scipy.signal import find_peaks, find_peaks_cwt, peak_prominences, periodogram, stft, peak_widths

In [4]:
def impute_df(df, whole_stats):
    nrows, ncols = df.shape
    nans_per_sensor = df.isna().sum(axis=0)
    for sensor in df.columns:
        n_nans = nans_per_sensor[sensor]
        if n_nans:
            mean = whole_stats.loc[sensor,"mean"]
            if n_nans == nrows:
                df[sensor].fillna(mean, inplace=True)
    
            else:
                data = df[sensor].values
                nan_mask = np.isnan(data)
                not_nan_indexes = np.argwhere(~nan_mask).squeeze()
                first_not_nan_index = not_nan_indexes[0]
                last_not_nan_index = not_nan_indexes[-1]
                nans_batch1 = nan_mask.copy()
                nans_batch1[last_not_nan_index+1:] = False
                data[nans_batch1] = mean
                
                # holt winters
                if last_not_nan_index != nrows-1:
                    #print(first_not_nan_index,last_not_nan_index)
                    autocorrelation = stattools.acf(data[:last_not_nan_index], nlags=40, fft=False)
                    higher_correlation = np.sort(autocorrelation)[-2]
                    period = np.argwhere(autocorrelation == higher_correlation).squeeze() + 1
                    holtwinters = ExponentialSmoothing(data[:last_not_nan_index], trend=None, seasonal="add", 
                                                       seasonal_periods=period, 
                                                       initialization_method="estimated").fit(optimized=True)
                    #print(last_not_nan_index)
                    forecasting = holtwinters.predict(start=last_not_nan_index+1, end=nrows-1)
                    #print(forecasting.shape)
                    #print(nan_mask.sum())
                    data[last_not_nan_index+1:] = forecasting
    
                df[sensor] = data
        
    return df

In [5]:
def estandarizar(df):
    aggs = df.agg([np.nanmean, np.nanstd]).astype("float16")
    standarized_df = (df - aggs.loc["nanmean",:])/ aggs.loc["nanstd",:]
    return standarized_df

In [7]:
def get_chars(df, nombre):
    cars = [int(nombre[:-4])]
    # zeros_crossings
    cars.extend(librosa.zero_crossings(df.values, axis = 0).sum(axis = 0))
    
    # find_peaks
    cars.extend(df.apply(find_peaks, axis = 0).iloc[0,:].apply(len).values)
            
    # peak_widths_max
    λ0 = lambda x: np.max(peak_widths(x, find_peaks(x)[0])[0]) if len(find_peaks(x)[0]) != 0 else 0
    cars.extend(df.apply(λ0).values)
                
    # peak_widths_mean
    λ01 = lambda x: np.mean(peak_widths(x, find_peaks(x)[0])[0]) if len(find_peaks(x)[0]) != 0 else 0
    cars.extend(df.apply(λ01).values)
                
    # peak_prominences_max
    λ1 = lambda x: np.max(peak_prominences(x, find_peaks(x)[0])[0]) if len(find_peaks(x)[0]) != 0 else 0
    cars.extend(df.apply(λ1).values)
                
    # peak_prominences_mean
    λ11 = lambda x: np.mean(peak_prominences(x, find_peaks(x)[0])[0]) if len(find_peaks(x)[0]) != 0 else 0
    cars.extend(df.apply(λ11).values)
                
    # periodogram_max
    λ2 = lambda x: np.max(periodogram(x, 100)[1]) if ~x.isna().all() else 0
    cars.extend(np.sqrt(df.apply(λ2).values)) # Es un estimado del RMS
    
    # periodogram_mean
    λ3 = lambda x: np.mean(periodogram(x, 100)[1]) if ~x.isna().all() else 0
    cars.extend(df.apply(λ3).values)
                
    return cars

In [8]:
whole_stats_path = "csvs/whole_stats.csv"
whole_stats = pd.read_csv(whole_stats_path, inddex_col=0)
whole_stats

In [ ]:
%%time
path_originales = ""
path_copias = "estandarizados"
for carpeta in ["train", "test"]:
    camino = path.join(path_originales, carpeta)
    print(f"Comienzo con {carpeta}")
    for file in listdir(camino):
        lectura = path.join(camino, file)
        escritura = path.join(path_copias, carpeta, file)
        df = pd.read_csv(lectura)
        df = impute_df(df)
        s_df = estandarizar(df).astype("Float16")
        s_df.to_csv(escritura, index = False)

In [ ]:
%%time

path_train = "estandarizados/train"
todos = []
i = 0
for archivo in listdir(path_train):
    camino = path.join(path_train, archivo)
    df = pd.read_csv(camino)
    try:
        chars = get_chars(df, archivo)
        todos.append(chars)
    except Exception as e:
        print(e,archivo)
    #print(i)
    i += 1
    
caracteristicas = "zcr peaks peak_wmax peak_wmean peak_promax peak_promean period_max period_mean". split()
cols = ["segment_id"] + [f"sensor_{idx}_{car}" for car in caracteristicas for idx in range(1,11)]
df_todos = pd.DataFrame(todos, columns = cols)
df_todos.to_csv("csvs/stats_per_file_signal_test.csv", index = False)